In [132]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict, Annotated
from langgraph.types import Send
import operator

In [133]:
from typing import Union


class State(TypedDict):
    words: list[str]
    output: Annotated[list[dict[str, Union[str, int]]], operator.add]


graph_builder = StateGraph(State)

In [134]:
def node_one(state: State):
    print(f"I want to count {len(state['words'])} words in my state.")
    return {}


def node_two(word: str):
    return {"output": [{"word": word, "letters": len(word)}]}

In [135]:
graph_builder.add_node("node_one", node_one)
graph_builder.add_node("node_two", node_two)


def dispatcher(state: State):
    return [Send("node_two", word) for word in state["words"]]


graph_builder.add_edge(START, "node_one")
graph_builder.add_conditional_edges("node_one", dispatcher, ["node_two"])
graph_builder.add_edge("node_two", END)

In [136]:
graph = graph_builder.compile()

graph.invoke({"words": ["hello", "world", "how", "are", "you", "doing"]})

I want to count 6 words in my state.


{'words': ['hello', 'world', 'how', 'are', 'you', 'doing'],
 'output': [{'word': 'hello', 'letters': 5},
  {'word': 'world', 'letters': 5},
  {'word': 'how', 'letters': 3},
  {'word': 'are', 'letters': 3},
  {'word': 'you', 'letters': 3},
  {'word': 'doing', 'letters': 5}]}